In [2]:
from failure_analysis_functions import load_traffic_centers, process_all_regions, save_results

In [3]:
#Traffic Center analysis

def run_flood_analysis():
    
    # Configuration
    traffic_centers_file = r"P:\bovenregionale-stresstest-hwn\Data\Traffic_centrals\Traffic_centers.xlsx"
    hazard_maps_base_path = r"P:\bovenregionale-stresstest-hwn\Data\Hazard_maps"
    output_directory = r"P:\bovenregionale-stresstest-hwn\Analysis\Traffic_Central_Analysis"
    region_list = ["ARK-NZK", "Vallei en Veluwe", "Noord-Westelijke Delta", "Noord-Brabant Oost"]
    buffer_distance = 1000  # meters
    
    # Step 1: Load and buffer traffic centers
    gdf_buffered = load_traffic_centers(traffic_centers_file, buffer_distance)
    
    # Step 2: Process all regions and flood maps
    gdf_with_floods, all_results = process_all_regions(gdf_buffered, region_list, hazard_maps_base_path)
    
    # Step 3: Save results
    gpkg_path, excel_path = save_results(gdf_with_floods, output_directory)
    
    # Generate summary statistics
    summary_stats = {
        'total_traffic_centers': len(gdf_with_floods),
        'centers_with_flood_data': (gdf_with_floods['max_flood_depth'] != -9999).sum(),
        'centers_with_flooding': (gdf_with_floods['max_flood_depth'] > 0).sum(),
        'max_flood_depth_found': gdf_with_floods['max_flood_depth'].max(),
        'mean_flood_depth': gdf_with_floods[gdf_with_floods['max_flood_depth'] != -9999]['max_flood_depth'].mean(),
        'gpkg_path': gpkg_path,
        'excel_path': excel_path
    }
    
    print(f"\nAnalysis complete!")
    print(f"Processed {len(all_results)} flood maps across {len(region_list)} regions.")
    
    # Return the three values that the calling code expects
    return gdf_with_floods, summary_stats, all_results
    
gdf_results, summary, results_list = run_flood_analysis()

Loading traffic centers...
Loaded 6 traffic centers with 1000m buffers

=== Processing region: ARK-NZK ===
Found 1 flood map files for region ARK-NZK
Processing ARKNZK-depth-maximum.tif...
Error processing geometry 1: Input shapes do not overlap raster.
Error processing geometry 3: Input shapes do not overlap raster.
Error processing geometry 5: Input shapes do not overlap raster.

=== Processing region: Vallei en Veluwe ===
Found 1 flood map files for region Vallei en Veluwe
Processing samengevoegd_max_inundatiediepte_5m.tif...
Error processing geometry 0: Input shapes do not overlap raster.
Error processing geometry 1: Input shapes do not overlap raster.
Error processing geometry 2: Input shapes do not overlap raster.
Error processing geometry 4: Input shapes do not overlap raster.
Error processing geometry 5: Input shapes do not overlap raster.

=== Processing region: Noord-Westelijke Delta ===

=== Processing region: Noord-Brabant Oost ===
Creating summary columns with maximum valu

CPLE_AppDefinedError: b"sqlite3_exec(UPDATE gpkg_ogr_contents SET feature_count = 6 WHERE lower(table_name) = lower('traffic_centers_with_flood_depths')) failed: disk I/O error"

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona\_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b"sqlite3_exec(UPDATE gpkg_ogr_contents SET feature_count = 6 WHERE lower(table_name) = lower('traffic_centers_with_flood_depths')) failed: disk I/O error"



Results saved to:
  - GeoPackage: P:\bovenregionale-stresstest-hwn\Analysis\Traffic_Central_Analysis\traffic_centers_with_flood_depths.gpkg
  - Excel: P:\bovenregionale-stresstest-hwn\Analysis\Traffic_Central_Analysis\traffic_centers_with_flood_depths.xlsx

Analysis complete!
Processed 2 flood maps across 4 regions.


In [ ]:
#On off ramp analysis
from pathlib import Path
import geopandas as gpd
from failure_analysis_functions import cluster_connected,aggregate_clusters_to_points

region_list = ["Achterhoek"]

for region in region_list:
    print(f"Processing region: {region} network")
    root_dir = Path(rf"P:\bovenregionale-stresstest-hwn\Analysis\{region}\Outputs")
    network_file = root_dir / "base_network_hazard.gpkg"
    network_gdf = gpd.read_file(network_file)
    ramps_gdf = network_gdf[network_gdf["BST_CODE_N"].isin(["AFR", "OPR"])]
    afr_gdf = ramps_gdf[ramps_gdf["BST_CODE_N"] == "AFR"].copy()
    opr_gdf = ramps_gdf[ramps_gdf["BST_CODE_N"] == "OPR"].copy()
    
    afr_gdf_clustered = cluster_connected(afr_gdf)
    output_gpkg = root_dir / "afr_LineSegments.gpkg"
    afr_gdf_clustered.to_file(output_gpkg, driver="GPKG")

    
    afr_gdf_aggregated = aggregate_clusters_to_points(afr_gdf_clustered, "EV1_ma", method="mean")
    output_gpkg_aggregated = root_dir / "afr_Points.gpkg"
    afr_gdf_aggregated.to_file(output_gpkg_aggregated, driver="GPKG")

    opr_gdf_clustered = cluster_connected(opr_gdf)
    output_gpkg = root_dir / "opr_LineSegments.gpkg"
    opr_gdf_clustered.to_file(output_gpkg, driver="GPKG")

    opr_gdf_aggregated = aggregate_clusters_to_points(opr_gdf_clustered, "EV1_ma", method="mean")
    output_gpkg_aggregated = root_dir / "opr_Points.gpkg"
    opr_gdf_aggregated.to_file(output_gpkg_aggregated, driver="GPKG")


Processing region: Achterhoek network


CPLE_AppDefinedError: b'sqlite3_exec(CREATE TABLE gpkg_extensions (table_name TEXT,column_name TEXT,extension_name TEXT NOT NULL,definition TEXT NOT NULL,scope TEXT NOT NULL,CONSTRAINT ge_tce UNIQUE (table_name, column_name, extension_name))) failed: table gpkg_extensions already exists'

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona\_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b'sqlite3_exec(CREATE TABLE gpkg_extensions (table_name TEXT,column_name TEXT,extension_name TEXT NOT NULL,definition TEXT NOT NULL,scope TEXT NOT NULL,CONSTRAINT ge_tce UNIQUE (table_name, column_name, extension_name))) failed: table gpkg_extensions already exists'


Index(['link_id', 'REF_ID', 'highway', 'avgspeed', 'lanes', 'id_NWB',
       'BST_CODE_N', 'NWSNAAM_HW', 'NETWERKSCH', 'node_A', 'node_B',
       'edge_fid', 'rfid_c', 'rfid', 'length', 'time', 'EV1_mi', 'EV1_ma',
       'EV1_me', 'EV1_fr', 'EV2_mi', 'EV2_ma', 'EV2_me', 'EV2_fr', 'geometry',
       'cluster_id'],
      dtype='object')


CPLE_AppDefinedError: b"sqlite3_exec(UPDATE gpkg_ogr_contents SET feature_count = 64 WHERE lower(table_name) = lower('afr_aggregated_test')) failed: unable to open database file"

Exception ignored in: 'fiona._shim.gdal_flush_cache'
Traceback (most recent call last):
  File "fiona\_err.pyx", line 201, in fiona._err.GDALErrCtxManager.__exit__
fiona._err.CPLE_AppDefinedError: b"sqlite3_exec(UPDATE gpkg_ogr_contents SET feature_count = 64 WHERE lower(table_name) = lower('afr_aggregated_test')) failed: unable to open database file"
